In [85]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re

import sys
sys.path.append("..")

import src.support.data_upload_support as dus


In [86]:
enlaces_navegacion = ["https://minimalismbrand.com/collections/hogar",
                      "https://minimalismbrand.com/collections/mochilas",
                      "https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a",
                      "https://minimalismbrand.com/collections/ropa-minimalism-mujer",
                      "https://minimalismbrand.com/collections/ropa-minimalism"
                      ]

In [87]:
def sacar_productos_pagina(soup, diccionario_extract, diccionario):
    try:
        product_cards_list = soup.find("div",{"class":"product-list product-list--per-row-3 product-list--per-row-mob-2 product-list--per-row-mob-2 product-list--image-shape-shortest"}).findChildren("div",recursive=False)
    except:
        print()
        return diccionario
    for product_card in product_cards_list:
        for campo, valor_func in diccionario_extract.items():
            try:
                diccionario[campo].append(valor_func(product_card))
        
            except:
                diccionario[campo].append(np.nan)
    return diccionario

Diccionarios funciones extraccion

In [88]:
diccionario_extract = {
    "nombre": lambda card: re.sub(r"\s+"," ",card.find("h3",{"class":"product-block__title"}).text.strip()),
    "precio": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[0].text.strip(),
    "precio_original": lambda card: card.find("div",{"class":"product-price"}).findChildren("span",recursive=False)[1].text.strip(),
    "url": lambda card: "https://minimalismbrand.com"+ card.find("a",{"class":"product-link"})["href"]
}


In [89]:
diccionario = {
        "nombre": [],
        "precio": [],
        "precio_original": [],
        "url": []
}
for enlace in enlaces_navegacion:
    print("Enlace principal", enlace)

    # 
    response = requests.get(enlace)
    
    soup = BeautifulSoup(response.text, 'html.parser')

    # enlaces en subcategoría
    enlace_subcategoria = soup.find("a",{"class":"image-overlay__image-link"})

    # SI ES PAGINA SUBCATEGORIAS
    if enlace_subcategoria:

        # ENCONTRAR ENLACES SUBCATEGORIAS, ENTRAR EN CADA PAGINA PRODUCTOS
        lista_enlace_pag_productos = ["https://minimalismbrand.com"+elemento["href"] for elemento in soup.find_all("a",{"class":"image-overlay__image-link"})]

        try:
            lista_nombres_productos = [elemento.find("span").text.strip() for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento and elemento.find("span")]
        except:
            # print("except")
            lista_nombres_productos = [elemento.find("span") for elemento in soup.find_all("a",{"class":"image-overlay__image-link"}) if elemento]

            
        # print(lista_nombres_productos)

        for enlace_pag_producto in lista_enlace_pag_productos: 
            # ENCONTRAR EN CADA URL PRODUCTO
            # print("Enlace_pag_producto", enlace_pag_producto)
            response = requests.get(enlace_pag_producto)
            subsoup = BeautifulSoup(response.text, 'html.parser')
            
            diccionario = sacar_productos_pagina(subsoup,diccionario_extract, diccionario)

    # SI ES PAGINA ENLACES A PRODUCTOS 
    else:
        diccionario = sacar_productos_pagina(soup,diccionario_extract, diccionario)

precios_minimalism = pd.DataFrame(diccionario)
precios_minimalism["precio"] = np.where(precios_minimalism["precio"]=="De", precios_minimalism["precio_original"], precios_minimalism["precio"])
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].fillna("-")
precios_minimalism[["precio","precio_original"]] = precios_minimalism[["precio","precio_original"]].map(lambda x: x.replace("€",""))
precios_minimalism.shape

Enlace principal https://minimalismbrand.com/collections/hogar
Enlace principal https://minimalismbrand.com/collections/mochilas
Enlace principal https://minimalismbrand.com/collections/camisetas-algodon-organico-nino-a
Enlace principal https://minimalismbrand.com/collections/ropa-minimalism-mujer

Enlace principal https://minimalismbrand.com/collections/ropa-minimalism



(258, 4)

In [90]:
precios_minimalism["url2"] = precios_minimalism["url"].str.split("?",expand=True)[0]
precios_minimalism.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   nombre           258 non-null    object
 1   precio           258 non-null    object
 2   precio_original  258 non-null    object
 3   url              258 non-null    object
 4   url2             258 non-null    object
dtypes: object(5)
memory usage: 10.2+ KB


In [91]:
precios_minimalism.drop_duplicates(subset="url2",inplace=True)
precios_minimalism.shape

(80, 5)

In [92]:
precios_minimalism

,nombre,precio,precio_original,url,url2
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...
6,Toalla de ducha · Algodón orgánico - Blanco,17.00,-,https://minimalismbrand.com/products/toalla-du...,https://minimalismbrand.com/products/toalla-du...
8,Toallas baño pack mediano · Algodón orgánico,24.00,26.00,https://minimalismbrand.com/products/toallas-b...,https://minimalismbrand.com/products/toallas-b...
9,Gel sólido uso diario x3,20.00,22.50,https://minimalismbrand.com/products/gel-solid...,https://minimalismbrand.com/products/gel-solid...
...,...,...,...,...,...
180,Calzoncillos Slips Orgánicos X3,32.00,36.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
181,Calzoncillos Slips Orgánicos X5,50.00,60.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
182,Calzoncillos Slips Orgánicos X7,68.00,84.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...
210,Pantalón orgánico Slim Hombre,70.00,-,https://minimalismbrand.com/products/pantalon-...,https://minimalismbrand.com/products/pantalon-...


In [165]:
precios_minimalism.to_csv("../datos/precios_minimalism.csv")

In [159]:
def extraer_info_producto(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text,"html.parser")

    # diccionario_campos = {
    #     "score": soup.find("span",{"class":"stamped-badge-caption"})["data-rating"].replace(",",".") if soup.find("span",{"class":"stamped-badge-caption"}) else None,
    #     "reviews": soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"] if soup.find("span",{"class":"stamped-badge-caption"}) else None,
    #     "vertical": " ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:]) if soup.find("div",{"class":"container pango-breadcrumb"}) else None,
    #     "description": soup.find('div', {"class":'cc-accordion'}).text.replace("\n","-") if soup.find('div', {"class":'cc-accordion'}) else None,
    #     "politica_envios": [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Política de envíos"][0] if [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Política de envíos"] else None,# if soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) else None,
    #     "impacto": [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Impacto ambiental"][0] if [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Impacto ambiental"] else None, # if len(soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'})) >=2 else None,
    #     "desglose_costes": [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Desglose de costes"][0] if [elemento.text.replace("\n","-") for elemento in soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) if elemento.find("summary").text == "Desglose de costes"] else None, # if len(soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'})) >=3 else None,
    #     "material2": soup.find('span', {"class":'with-icon__beside'}).text if soup.find('span', {"class":'with-icon__beside'}) else None,
    #     "cultivo":[element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[0].findChildren()] if soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"}) else None,
    #     "confeccion": [element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[1].findChildren()]  if len(soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})) > 1 else None,
    #     "logistica": [element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[2].findChildren()]  if len(soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})) > 2 else None
        
    # }

    diccionario_campos = {
        "score": lambda: soup.find("span", {"class": "stamped-badge-caption"})["data-rating"].replace(",", "."),
        "reviews": lambda: soup.find("span", {"class": "stamped-badge-caption"})["data-reviews"],
        "vertical": lambda: " ".join(soup.find("div", {"class": "container pango-breadcrumb"}).text.strip().split()[2:]),
        "description": lambda: soup.find("div", {"class": "cc-accordion"}).text.replace("\n", "-"),
        "politica_envios": lambda: [elemento.text.replace("\n", "-") for elemento in soup.findAll('div', {"class": "product-detail-accordion not-in-quickbuy"}) if elemento.find("summary").text == "Política de envíos"][0],
        "impacto": lambda: [elemento.text.replace("\n", "-") for elemento in soup.findAll('div', {"class": "product-detail-accordion not-in-quickbuy"}) if elemento.find("summary").text == "Impacto ambiental"][0],
        "desglose_costes": lambda: [elemento.text.replace("\n", "-") for elemento in soup.findAll('div', {"class": "product-detail-accordion not-in-quickbuy"}) if elemento.find("summary").text == "Desglose de costes"][0],
        "material2": lambda: soup.find('span', {"class": 'with-icon__beside'}).text,
        "cultivo": lambda: " # ".join([element.text for element in soup.find("div", {"class": "flexible-layout"}).find_all("div", {"class": "column"})[0].findChildren()]),
        "confeccion": lambda: " # ".join([element.text for element in soup.find("div", {"class": "flexible-layout"}).find_all("div", {"class": "column"})[1].findChildren()]),
        "logistica": lambda: " # ".join([element.text for element in soup.find("div", {"class": "flexible-layout"}).find_all("div", {"class": "column"})[2].findChildren()])
    }



    diccionario_rellenar = {
        "score": None,
        "reviews": None,
        "vertical": None,
        "description": None,
        "politica_envios": None,
        "impacto": None,
        "desglose_costes": None,
        "material2": None,
        "cultivo": None,
        "confeccion": None,
        "logistica": None
    }


    for key, function_dict in diccionario_campos.items():
        try:
            diccionario_rellenar[key] = function_dict()
        except:
            diccionario_rellenar[key] = np.nan

    return [*diccionario_rellenar.values()]


In [160]:
url = "https://minimalismbrand.com/products/juego-de-sabanas-de-algodon-organico?variant=47272870773071"

In [161]:
precios_minimalism["url2"].apply(lambda x: extraer_info_producto(x)).to_list()

[['3.7',
  '3',
  'Esenciales para tu hogar',
  '--Descripción--\xa0-Ropa de cama suave al tacto\xa0confeccionada con\xa0100% algodón orgánico. Al emplear algodón orgánico\xa0reducimos el\xa0consumo de agua y energía, a la vez que evitamos\xa0la aplicación de químicos\xa0en terrenos de cultivo.-Confeccionados de forma consciente y sostenible en el norte de Portugal, en la ciudad de Fafe. Nuestro taller inició su actividad en el año 2000 y cuenta con certificado GOTS en hilo.-\xa0-¿Qué incluye cada juego de sábanas?-105x200: --Sábana bajera:\xa0 105 x 200 x 30 cm--Sábana encimera:\xa0 180 x 290\xa0cm--x1 Fundas almohada:\xa0 50 x 70 cm---\xa0-135x200:--Sábana bajera:\xa0 135 x 200 x 30 cm--Sábana encimera:\xa0  220\xa0x 290\xa0cm--x2 Fundas almohada:\xa0 50 x 70 cm----150x200:--Sábana bajera:\xa0 150 x 200 x 30 cm--Sábana encimera:\xa0 240\xa0x 290\xa0cm--x2 Fundas almohada:\xa0 50 x 70 cm---\xa0-180x200:--Sábana bajera:\xa0 180 x 200 x 30 cm--Sábana encimera:\xa0 260\xa0x 290\xa0cm--x2

In [162]:
precios_minimalism_short = precios_minimalism.iloc[0:2,:]

In [163]:
precios_minimalism_short[["score", "reviews", "vertical", "description", "politica_envios", "impacto", "desglose_costes", "material2", "cultivo", "confeccion", "logistica"]] = precios_minimalism_short["url2"].apply(lambda x: extraer_info_producto(x)).to_list()

C:\Users\Miguel\AppData\Local\Temp\ipykernel_22120\4115439786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precios_minimalism_short[["score", "reviews", "vertical", "description", "politica_envios", "impacto", "desglose_costes", "material2", "cultivo", "confeccion", "logistica"]] = precios_minimalism_short["url2"].apply(lambda x: extraer_info_producto(x)).to_list()


In [164]:
precios_minimalism_short

,nombre,precio,precio_original,url,url2,score,reviews,vertical,description,politica_envios,impacto,desglose_costes,material2,cultivo,confeccion,logistica
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...,3.7,3,Esenciales para tu hogar,--Descripción-- -Ropa de cama suave al tacto c...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSTICO Y MANUFACTURA # Almacén / ...
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...,4.2,9,Esenciales para tu baño,--Descripción---Toallas de rizo absorbente y s...,---Política de envíos--Envío a España y Portug...,NaN,---Desglose de costes--El precio final de este...,NaN,1. CULTIVO DEL ALGODÓN # Procedencia del algod...,2. CONFECCIÓN Y MANUFACTURA # Algodón certific...,3. CENTRO LOGÍSTICO Y MANUFACTURA # Almacén / ...


In [ ]:
precios_minimalism

,nombre,precio,precio_original,url,url2,score,reviews,vertical,description,politica_envios,impacto,desglose_costes,material2,cultivo,confeccion,logistica
0,Juego de sábanas · Algodón orgánico - 105x200,70.00,-,https://minimalismbrand.com/products/juego-de-...,https://minimalismbrand.com/products/juego-de-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Toalla lavabo · Algodón orgánico - Blanco,9.00,-,https://minimalismbrand.com/products/toalla-la...,https://minimalismbrand.com/products/toalla-la...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Toalla de ducha · Algodón orgánico - Blanco,17.00,-,https://minimalismbrand.com/products/toalla-du...,https://minimalismbrand.com/products/toalla-du...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Toallas baño pack mediano · Algodón orgánico,24.00,26.00,https://minimalismbrand.com/products/toallas-b...,https://minimalismbrand.com/products/toallas-b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Gel sólido uso diario x3,20.00,22.50,https://minimalismbrand.com/products/gel-solid...,https://minimalismbrand.com/products/gel-solid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,Calzoncillos Slips Orgánicos X3,32.00,36.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,Calzoncillos Slips Orgánicos X5,50.00,60.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182,Calzoncillos Slips Orgánicos X7,68.00,84.00,https://minimalismbrand.com/products/calzoncil...,https://minimalismbrand.com/products/calzoncil...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
210,Pantalón orgánico Slim Hombre,70.00,-,https://minimalismbrand.com/products/pantalon-...,https://minimalismbrand.com/products/pantalon-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [166]:
precios_minimalism.to_csv("../datos/precios_minimalism.csv")

In [167]:
dus.upload_to_drive("../datos/precios_minimalism.csv", '1vfttgGDHa3-eCMEN79QAue6aI3Gc1vvp')

SSLEOFError: EOF occurred in violation of protocol (_ssl.c:2423)

## Pruebas beautiful soup pagina producto

In [15]:
url_pag_producto = "https://minimalismbrand.com/products/camiseta-algodon-organico?variant=34948534108317"
response = requests.get(url_pag_producto)

soup = BeautifulSoup(response.text, 'html.parser')

Conseguir vertical

In [16]:
" ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:])

'Camisetas algodón orgánico'

In [ ]:
def extraer_info_producto(url):
    response = requests.get(url)

    soup = BeautifulSoup(response.text,"html.parser")

    diccionario_campos = {
        "score": soup.find("span",{"class":"stamped-badge-caption"})["data-rating"].replace(",",".") if soup.find("span",{"class":"stamped-badge-caption"}) else None,
        "reviews": soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"] if soup.find("span",{"class":"stamped-badge-caption"}) else None,
        "vertical": " ".join(soup.find("div",{"class":"container pango-breadcrumb"}).text.strip().split()[2:]) if soup.find("div",{"class":"container pango-breadcrumb"}) else None,
        "description": soup.find('div', {"class":'cc-accordion'}).text.replace("\n","-") if soup.find('div', {"class":'cc-accordion'}) else None,
        "politica_envios": soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'})[0].text.replace("\n","-") if soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) else None,
        "impacto": soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'})[1].text.replace("\n","-") if soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) else None,
        "desglose_costes": soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'})[2].text.replace("\n","-") if soup.findAll('div', {"class":'product-detail-accordion not-in-quickbuy'}) else None,
        "material2": soup.find('span', {"class":'with-icon__beside'}).text if soup.find('span', {"class":'with-icon__beside'}) else None,
        "cultivo":[element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[0].findChildren()],
        "confeccion": [element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[1].findChildren()],
        "logistica": [element.text for element in soup.find("div",{"class":"flexible-layout"}).find_all("div",{"class":"column"})[2].findChildren()]
        
    }

    return [*diccionario_campos.values()]



In [ ]:
precios_minimalism[["score","reviews","vertical","description","politica_envios","impacto","desglose_costes", "material2","cultivo","confeccion","logistica"]] = precios_minimalism["url2"].apply(lambda x: extraer_info_producto(x)).to_list()

Conseguir score

In [ ]:
score = soup.find("span",{"class":"stamped-badge-caption"})["data-rating"]


Conseguir reviews

In [ ]:
n_reviews = soup.find("span",{"class":"stamped-badge-caption"})["data-reviews"]